In [1]:
import pandas as pd
import requests
import json

In [79]:
from networkx.readwrite.json_graph import node_link_data, node_link_graph
from networkx.classes.function import degree

Функції для роботи з графом:

In [80]:
def read_json(json_name):
    with open(json_name, 'r') as infile:
        data = json.load(infile)
    g = node_link_graph(data, directed=True) # directed or undirected
    return g

In [81]:
def write_json(g, json_name, graph=True):
    if graph:
        g = node_link_data(g)
    with open(json_name, 'w') as outfile:
        json.dump(g, outfile)

In [82]:
def nodes_degree(g):
    dct = dict(degree(g.to_undirected()))
    for i in dct:
        g.nodes[i]['degree'] = dct[i]
    return g

Функції для роботи з парсингом:

In [2]:
def req(obj): # витягуємо дані ЄДР із сайту ring.org.ua для даного об'єкта
    url = 'https://ring.org.ua/search?q=' + obj + '&datasources=edrdr&format=json'
    response = requests.get(url) # надсилаємо get-запит на сервер ring.org.ua
    #response = requests.get(url, verify=False) # якщо матюкається на SSL-сертифікат
    if response.status_code == 200:
        dict_resp = response.json()
        items = dict_resp['search_results']['paginator']['count'] # кількість пов'язаних фірм
        pages = dict_resp['search_results']['paginator']['num_pages'] # кількість сторінок у результаті
        result = dict_resp['search_results']['object_list'] # формуємо список даних про фірми
        if pages > 1: # якщо результати запиту не вмістилися на одну сторінку (не може бути більше 1000?)
            for i in range(2, pages+1): # повторюємо запит для всіх сторінок, починаючи з другої
                url = 'https://ring.org.ua/search?q=' + obj + '&datasources=edrdr&format=json&page=' + str(i)
                response = requests.get(url) # надсилаємо get-запит на сервер ring.org.ua
                #response = requests.get(url, verify=False) # якщо матюкається на SSL-сертифікат
                if response.status_code == 200:
                    dict_resp = response.json()
                    result += dict_resp['search_results']['object_list'] # поповнюємо список даних про фірми
                else:
                    print('Error: requested url ' + url + ' is not available!')
        if len(result) == items: # перевіряємо, чи нам вдалося отримати інформацію про всі пов'язані фірми
            return (items, result)
        else:
            print('Warning: parsed url ' + url.split('&page=')[0] + ' is not full!')
            return (len(result), result)
    else:
        print('Error: requested url ' + url + ' is not available!')
        return (0, ['error'])

In [3]:
def reg(loc): # визначаємо номер регіону за повною адресою фірми
    if (loc.find('київська область') != -1) or (loc.find('київська обл.') != -1):
        return 1
    elif (loc.find('дніпропетровська область') != -1) or (loc.find('дніпропетровська обл.') != -1):
        return 2
    elif (loc.find('донецька область') != -1) or (loc.find('донецька обл.') != -1):
        return 3
    elif (loc.find('харківська область') != -1) or (loc.find('харківська обл.') != -1):
        return 4
    elif (loc.find('одеська область') != -1) or (loc.find('одеська обл.') != -1):
        return 5
    elif (loc.find('львівська область') != -1) or (loc.find('львівська обл.') != -1):
        return 6
    elif (loc.find('миколаївська область') != -1) or (loc.find('миколаївська обл.') != -1):
        return 7
    elif (loc.find('запорізька область') != -1) or (loc.find('запорізька обл.') != -1):
        return 8
    elif (loc.find('хмельницька область') != -1) or (loc.find('хмельницька обл.') != -1):
        return 9
    elif (loc.find('полтавська область') != -1) or (loc.find('полтавська обл.') != -1):
        return 10
    elif (loc.find('луганська область') != -1) or (loc.find('луганська обл.') != -1):
        return 11
    elif (loc.find('вінницька область') != -1) or (loc.find('вінницька обл.') != -1):
        return 12
    elif (loc.find('житомирська область') != -1) or (loc.find('житомирська обл.') != -1):
        return 13
    elif (loc.find('черкаська область') != -1) or (loc.find('черкаська обл.') != -1):
        return 14
    elif (loc.find('херсонська область') != -1) or (loc.find('херсонська обл.') != -1):
        return 15
    elif (loc.find('кіровоградська область') != -1) or (loc.find('кіровоградська обл.') != -1):
        return 16
    elif (loc.find('сумська область') != -1) or (loc.find('сумська обл.') != -1):
        return 17
    elif (loc.find('івано-франківська область') != -1) or (loc.find('івано-франківська обл.') != -1):
        return 18
    elif (loc.find('тернопільська область') != -1) or (loc.find('тернопільська обл.') != -1):
        return 19
    elif (loc.find('чернігівська область') != -1) or (loc.find('чернігівська обл.') != -1):
        return 20
    elif (loc.find('волинська область') != -1) or (loc.find('волинська обл.') != -1):
        return 21
    elif (loc.find('рівненська область') != -1) or (loc.find('рівненська обл.') != -1):
        return 22
    elif (loc.find('закарпатська область') != -1) or (loc.find('закарпатська обл.') != -1):
        return 23
    elif (loc.find('чернівецька область') != -1) or (loc.find('чернівецька обл.') != -1):
        return 24
    elif (loc.find('республіка крим') != -1) or (loc.find(' арк ') != -1) or (loc.find(' арк,') != -1):
        return 25
    elif (loc.find(' київ ') != -1) or (loc.find(' київ,') != -1):
        return 1
    elif (loc.find(' севастополь ') != -1) or (loc.find(' севастополь,') != -1):
        return 25
    else:
        return 0 # адреса не підходить під жодну з наших масок, тому на неї далі не звертатимемо уваги

In [4]:
def find_addr(addresses_cut, records_cut, person): # знаходимо список адрес бенефіціара
    list_addr = []
    for address in addresses_cut:
        for record in records_cut:
            if record.find(person.replace(' ', '').replace('-', '')) != -1 and record.find(address) != -1:
                list_addr.append(address)
    return list(set(list_addr)) # видаляємо дублі, які могли утворитися протягом обробки

In [5]:
def add_info(firm, info=0, list_addr=None): # головна підпрограма -- обробляє картку даних однієї фірми
    next_step = False
    # додаємо фірму до словника знайдених контрагентів
    edrpou = firm['full_edrpou']
    name = firm['latest_record']['name'].lower().replace('  ', ' ')
    short_name = firm['latest_record']['short_name'].lower().replace('  ', ' ')
    status = firm['latest_record']['status'].lower().replace('  ', ' ') 
    dict_find[edrpou] = (name, short_name, status) # додаємо інформацію про фірму: назва, повна назва, статус
    dict_firm[name] = edrpou # якщо будуть дублі за повною назвою фірми, то запишеться лише останній ЄДРПОУ!
    if status != 'припинено': # якщо фірма функціонує
        try:
            records = firm['raw_records'] # тут міститься інформація про частки участі засновників (у грн.)
            if type(info) == str: # перевірка № 1 -- за співпадінням назв фірм
                if info in [record.split(',')[0].lower().replace('  ', ' ') for record in records]:
                    location = firm['latest_record']['location'].lower().replace('  ', ' ')
                    region = reg(location) # знаходимо регіон, де розташована фірма
                    next_step = True
            else: # перевірка № 2 -- за співпадінням адрес при спільному П.І.Б. бенефіціарів
                if list_addr != None and len(list([person[0] for person in firm['raw_persons'] 
                                                   if person[1] == 'Бенефіціарний власник'])) > 0: # є адреса і бенефіціари
                    for addr in list_addr: # перебираємо список адрес (можуть відрізнятися незначним чином)
                        if not next_step:
                            for record in records: # для кожного запису про частку участі засновника
                                if record.lower().replace(' ', '').replace('-', '').find(addr) != -1: # адреси співпали
                                    location = firm['latest_record']['location'].lower().replace('  ', ' ')
                                    region = reg(location) # знаходимо регіон, де розташована фірма
                                    next_step = True
                                    break
                else: # перевірка № 3 -- за співпадінням регіонів фірм при спільному П.І.Б. засновників
                    location = firm['latest_record']['location'].lower().replace('  ', ' ')
                    region = reg(location) # знаходимо регіон, де розташована фірма
                    if (info == 0) or (info == region): # фірми з іншого регіону не розглядаємо (засновники -- однофамільці?)
                        next_step = True
        except:
            print('Warning: Detail information about firm {0} is absend, status = {1}.'.format(edrpou, status))
    if next_step:
        # якщо ця фірма ще не включена до списку пошуку (поточного або наступного)
        if (name not in [x[0] for x in list_temp]) and (name not in [x[0] for x in list_next]):
        #if (name not in list_temp) and (name not in list_next): # якщо ця фірма ще не включена до списку пошуку
            list_next.append((name, -1)) # на наступному рівні шукатимемо засновані цією фірмою фірми
            # додаємо інформацію про засновників фірми
            list_all = list(set([person[0].lower().replace('  ', ' ') for person in firm['raw_persons']])) # фізособи
            list_master = list(set([person[0].lower().replace('  ', ' ') for person in firm['raw_persons'] 
                                    if person[1] == 'Голова'])) # список голів фірми (голова тільки один?)
            list_founder = list(set([person[0].lower().replace('  ', ' ') for person in firm['raw_persons'] 
                                     if person[1] == 'Засновник'])) # список засновників (вносили кошти)
            list_benef = list(set([person[0].lower().replace('  ', ' ') for person in firm['raw_persons'] 
                                   if person[1] == 'Бенефіціарний власник'])) # список бенефіціарів
            if len(list_benef) > 0: # адреси засновників вказуються зазвичай тільки для бенефіціарів
                addresses = firm['addresses']
                addresses = [address.lower().replace(' ', '').replace('-', '') for address in addresses if 
                             len(address.split(',')) > 1] # вилучаємо всі пробіли й дефіси та адреси з одного слова
                records_new = [record.lower().replace(' ', '').replace('-', '') for record in records]
                addresses_cut = []
                for address in addresses: # лишаємо ті адреси, які є в записах про частку участі засновників
                    if sum([False if record.find(address) == -1 else True for record in records_new]) > 0:
                        addresses_cut.append(address)
                addresses_cut = list(set(addresses_cut)) # видаляємо дублі, які могли утворитися протягом обробки
                records_cut = []
                for record in records_new: # лишаємо тільки ті записи про частку участі, у яких є адреси
                    if sum([False if record.find(address) == -1 else True for address in addresses_cut]) > 0:
                        records_cut.append(record)
            for person in list_all:
                if person not in set(dict_find.keys()): # якщо ця особа ще не засвітилася раніше
                    dict_find[person] = () # додаємо її П.І.Б. до словника знайдених контрагентів
                    # і якщо ця особа не включена до списку пошуку (поточного або наступного)
                    if (person not in [x[0] for x in list_temp]) and (person not in [x[0] for x in list_next]): 
                    #if (person not in list_temp) and (person not in list_next): # і якщо вона не включена до пошуку
                        if person not in list_benef: # список бенефіціарів може бути порожнім
                            list_next.append((person, region)) # на наступному рівні шукатимемо засновані цією особою фірми
                        else: # якщо ми маємо справу з бенефіціаром, додаємо також його адресу
                            list_next.append((person, region, find_addr(addresses_cut, records_cut, person)))
            for head in list_master: # додаємо інформацію про голів фірм, вони можуть не бути засновниками
                list_head.append([head, edrpou])
            part = dict()
            for record in records: # відбираємо інформацію тільки про ненульові внески
                if (record.find(' грн.')!=-1) and (record.find(' 0.00 грн.')==-1) and (record.find(' 0,00 грн.')==-1):
                    part[record.split(',')[0].lower().replace('  ', ' ')] = float(
                        record.split('-')[-1].split('грн')[0].strip().replace(',', '.')) # засновник і його внесок
            if len(part) > 0: # якщо є інформація хоча б про один ненульовий внесок до статутного капіталу
                capital = sum(part.values())
                for founder in list(part):
                    if founder in list_all: # засновник -- фізична особа
                        if founder in list_benef: # засновник є бенефіціаром
                            list_type.append([founder, edrpou, part[founder]/capital, 1])
                        else: # засновник не є бенефіціаром
                            list_type.append([founder, edrpou, part[founder]/capital, 0])
                    else: # засновник -- юридична особа (фірма)
                        if founder in list(dict_firm): # і для цієї фірми ми вже маємо інформації про її ЄДРПОУ
                            list_type.append([dict_firm[founder], edrpou, part[founder]/capital, 0])
                        else: # інакше записуємо просто назву фірми
                            list_type.append([founder, edrpou, part[founder]/capital, 0])
            # якщо бенефіціар не вносив кошти до статутного капіталу, додаємо інформацію про нього
            zero_benef = list(set(list_benef) - set(list(part)))
            if len(zero_benef) > 0:
                for founder in zero_benef:
                    list_type.append([founder, edrpou, 0, 1])
            # якщо засновник не є бенефіціаром і не вносив кошти до статутного капіталу, додаємо інформацію про нього
            zero_founder = list(set(list_founder) - set(list_benef) - set(list(part)))
            if len(zero_founder) > 0:
                for founder in zero_founder:
                    list_type.append([founder, edrpou, 0, 0])

In [6]:
dict_find = dict() # словник знайдених контрагентів
dict_firm = dict() # словник назва фірми --> ЄДРПОУ
list_type = [] # список зв'язків між знайденими контрагентами
list_head = [] # список голів фірм
list_temp = [] # список контрагентів на поточному рівні
list_next = [] # список контрагентів наступного рівня -- кортежі: назва фірми або П.І.Б., регіон (-1 для фірми)
i = 0 # лічильник (номер рівня відносно початкового об'єкта)

In [7]:
# ці дані буде задавати користувач під час запуску
start = '31382665' # задаємо ЄДРПОУ початкової фірми
level = 3 # задаємо максимальну кількість рівнів

In [8]:
%%time
list_next.append((start, 0)) # 0 -- для вказівки на те, що нам не важливо, з якого регіону дана фірма
while (len(list_next) > 0) and (i < level):
    list_temp = list_next.copy()
    list_next = []
    for obj in list_temp:
        resp = req(obj[0])
        num = resp[0] # кількість знайдених фірм, пов'язаних із даним об'єктом (ЄДРПОУ, назва фірми, П.І.Б.)
        if num != 0:
            list_obj = resp[1] # список даних по знайдених фірмах
            for n in range(num):
                edrpou = list_obj[n]['full_edrpou'] # підтягуємо ЄДРПОУ фірми (що робити, якщо його немає?)
                if edrpou not in set(dict_find.keys()): # якщо ми ще не обробляли інформацію про дану фірму
                    if obj[1] == -1: # додаємо інформацію про засновників фірми та нові об'єкти для пошуку
                        add_info(list_obj[n], obj[0]) # тут передаємо для перевірки рядок -- назву фірми
                    elif len(obj) == 2: # додаємо те ж саме, тільки для фірм із певної області
                        add_info(list_obj[n], obj[1]) # тут передаємо ціле число -- номер регіону (або 0)
                    else:
                        add_info(list_obj[n], obj[1], obj[2]) # а тут додаємо ще й адресу засновника
        else:
            if resp[1] == ['error']:
                print('Error request for object = ', obj[0])
    i += 1

CPU times: user 3.31 s, sys: 129 ms, total: 3.44 s
Wall time: 35.1 s


In [9]:
dict_find # знайдені контрагенти (вершини графа), для фірм додаємо повну назву, скорочену назву і стан

{'31382665': ('товариство з обмеженою відповідальністю "сигарний дім фортуна"',
  'тов "сигарний дім фортуна"',
  'зареєстровано'),
 'білоус вікторія віталіївна': (),
 'білоус віталій григорійович': (),
 'єлісєєв олександр': (),
 '42275040': ('товариство з обмеженою відповідальністю "хіт-трейд"',
  'тов "хіт-трейд"',
  'зареєстровано'),
 'погорєлов вячеслав іванович': (),
 'погорелов вячеслав іванович': (),
 '35050706': ('товариство з обмеженою відповідальністю "фортуна інвест"',
  'тов "фортуна інвест"',
  'зареєстровано'),
 'хотякова олена': (),
 'бадюл ігор анатолійович': (),
 'чайковський сергій олександрович': (),
 'хотякова неллі євгеніївна': (),
 'хаіт олександр леонідович': (),
 '39892402': ('товариство з обмеженою відповідальністю "лакант"',
  'тов "лакант"',
  'припинено'),
 '35818426': ('приватне підприємство "атп-15101"',
  'пп"атп-15101"',
  'зареєстровано'),
 '42229743': ('товариство з обмеженою відповідальністю "автопарк-юг"',
  'тов "автопарк-юг"',
  'зареєстровано'),
 

In [10]:
dict_firm # словник для визначення ЄДРПОУ фірми за її назвою (сподіваюсь, дублів за повними назвами фірм немає)

{'товариство з обмеженою відповідальністю "сигарний дім фортуна"': '31382665',
 'товариство з обмеженою відповідальністю "хіт-трейд"': '42275040',
 'товариство з обмеженою відповідальністю "фортуна інвест"': '35050706',
 'товариство з обмеженою відповідальністю "лакант"': '39892402',
 'приватне підприємство "атп-15101"': '35818426',
 'товариство з обмеженою відповідальністю "автопарк-юг"': '42229743',
 'приватне підприємство "аврора"': '32478379',
 'товариство з обмеженою відповідальністю "аріадна."': '24526422',
 'товариство з обмеженою відповідальністю "рент-ол"': '42557980',
 'товариство з обмеженою відповідальністю "актив естейт"': '41424397',
 'товариство з обмеженою відповідальністю "елітстройпроект"': '34873867',
 'товариство з обмеженою відповідальністю "рент-естейт"': '41680460',
 'товариство з обмеженою відповідальністю "проект-ф"': '43376508',
 'товариство з обмеженою відповідальністю "спот юг"': '41365887',
 'товариство з обмеженою відповідальністю "ленд юг"': '42771752',
 

In [11]:
len(dict_firm)

197

In [12]:
list_type # (засновник, ЄДРПОУ заснованої фірми, частка участі засновника, чи є засновник бенефіціаром)

[['єлісєєв олександр', '31382665', 0.5, 1],
 ['білоус вікторія віталіївна', '31382665', 0.5, 1],
 ['31382665', '42275040', 1.0, 0],
 ['білоус вікторія віталіївна', '42275040', 0, 1],
 ['єлісєєв олександр', '35050706', 0.2539348392481973, 1],
 ['хотякова олена', '35050706', 0.13075368759157163, 0],
 ['бадюл ігор анатолійович', '35050706', 0.15382786858359165, 1],
 ['хаіт олександр леонідович', '35050706', 0.15382786858359165, 0],
 ['хотякова неллі євгеніївна', '35050706', 0.13075368759157163, 0],
 ['білоус вікторія віталіївна', '35050706', 0.17690204840147622, 1],
 ['зарх микита', '24526422', 0.028204235166345293, 1],
 ['бадюл ігор анатолійович', '24526422', 0.08285007025437408, 1],
 ['білоус віталій григорійович', '24526422', 0.32989358206736075, 0],
 ['хаіт олександр леонідович', '24526422', 0.09545156594006436, 0],
 ['єлісєєв олександр', '24526422', 0.32989358206736075, 0],
 ['беллкор інвестмент лімітед (bellkor investment limited)',
  '24526422',
  0.0038587420220974724,
  0],
 ['за

Якщо ЄДРПОУ по якихось фірмах не підтягнулося відразу, то заміняємо зараз назву фірми на ЄДРПОУ (заміняється не завжди):

In [13]:
%%time
list_type_new = list_type.copy()
for i in range(len(list_type_new)):
    if list_type_new[i][0] in list(dict_firm):
        list_type_new[i][0] = dict_firm[list_type_new[i][0]]
list_type_new

CPU times: user 3.33 ms, sys: 0 ns, total: 3.33 ms
Wall time: 3.23 ms


[['єлісєєв олександр', '31382665', 0.5, 1],
 ['білоус вікторія віталіївна', '31382665', 0.5, 1],
 ['31382665', '42275040', 1.0, 0],
 ['білоус вікторія віталіївна', '42275040', 0, 1],
 ['єлісєєв олександр', '35050706', 0.2539348392481973, 1],
 ['хотякова олена', '35050706', 0.13075368759157163, 0],
 ['бадюл ігор анатолійович', '35050706', 0.15382786858359165, 1],
 ['хаіт олександр леонідович', '35050706', 0.15382786858359165, 0],
 ['хотякова неллі євгеніївна', '35050706', 0.13075368759157163, 0],
 ['білоус вікторія віталіївна', '35050706', 0.17690204840147622, 1],
 ['зарх микита', '24526422', 0.028204235166345293, 1],
 ['бадюл ігор анатолійович', '24526422', 0.08285007025437408, 1],
 ['білоус віталій григорійович', '24526422', 0.32989358206736075, 0],
 ['хаіт олександр леонідович', '24526422', 0.09545156594006436, 0],
 ['єлісєєв олександр', '24526422', 0.32989358206736075, 0],
 ['беллкор інвестмент лімітед (bellkor investment limited)',
  '24526422',
  0.0038587420220974724,
  0],
 ['за

In [14]:
list_head # (голова, ЄДРПОУ фірми)

[['білоус віталій григорійович', '31382665'],
 ['погорєлов вячеслав іванович', '42275040'],
 ['погорелов вячеслав іванович', '42275040'],
 ['чайковський сергій олександрович', '35050706'],
 ['звєрєв сергій валерійович', '24526422'],
 ['будницький максим валерійович', '42557980'],
 ['кручко галина володимирівна', '42557980'],
 ['мартиненко михайло сергійович', '41424397'],
 ['будницька катерина сергіївна', '34873867'],
 ['пасечник ганна олексіївна', '34873867'],
 ['затона максим миколайович', '41680460'],
 ['будницька катерина сергіївна', '43376508'],
 ['сокол алла олександрівна', '41365887'],
 ['затона максим миколайович', '42771752'],
 ['будяну олександр георгійович', '41196618'],
 ['затона максим миколайович', '41196618'],
 ['сокол алла олександрівна', '36503216'],
 ['ніколаєв роман вячеславович', '42084769'],
 ['волянська наталія борисівна', '42084769'],
 ['волянська євгенія олександрівна', '42084769'],
 ['волкова надія анатоліївна', '42084769'],
 ['циганова олена василівна', '41133

In [15]:
list_temp # список контрагентів, за якими здійснювався пошук на останньому рівні

[('товариство з обмеженою відповідальністю "хіт-трейд"', -1),
 ('погорєлов вячеслав іванович', 5),
 ('погорелов вячеслав іванович', 5),
 ('товариство з обмеженою відповідальністю "фортуна інвест"', -1),
 ('хотякова олена', 5),
 ('бадюл ігор анатолійович', 5, []),
 ('чайковський сергій олександрович', 5),
 ('хотякова неллі євгеніївна', 5),
 ('хаіт олександр леонідович', 5),
 ('товариство з обмеженою відповідальністю "аріадна."', -1),
 ('зарх микита', 5, ['одеськаобласть,м.одеса,приморськийбульвар.буд.10,кв.5']),
 ('зарх данііл', 5),
 ('звєрєв сергій валерійович', 5),
 ('зарх владислав', 5),
 ('товариство з обмеженою відповідальністю "рент-ол"', -1),
 ('будницький максим валерійович', 5),
 ('кручко галина володимирівна', 5),
 ('товариство з обмеженою відповідальністю "актив естейт"', -1),
 ('мартиненко михайло сергійович', 5),
 ('зарх даніїл', 5),
 ('кочергіна світлана олександрівна',
  5,
  ['житомирськаобл.,м.бердичів,вул.житомирськабуд.51,кв.47',
   'вн,житомирськаобл.,м.бердичів,вул.

In [16]:
list_next # список контрагентів для пошуку на наступному рівні (непорожній, якщо було задано максимальний рівень)

[('товариство з обмеженою відповідальністю "укрбізнесавто"', -1),
 ('сироткіна ірина георгіївна', 5),
 ('приватне підприємство "управитель"', -1),
 ('крамаренко вікторія георгіївна', 5),
 ('громадська організація "одеський музей авіаційної техніки"', -1),
 ('оробей сергій ігорович', 5),
 ('дівєєв олександр володимирович', 5),
 ('наливайко валерій валентинович', 5),
 ('більтяєв олексій юрійович', 5),
 ('товариство з обмеженою відповідальністю "бєнєфіт ав"', -1),
 ('хвостова тетяна михайлівна', 5),
 ('приватне підприємство "маранта плюс"', -1),
 ('заволокіна олена григорівна', 5),
 ('товариство з обмеженою відповідальністю "ленд естейт"', -1),
 ('заїчко інна олександрівна', 5),
 ('стаматова неля іванівна', 5),
 ('товариство з обмеженою відповідальністю "льодова зірка"', -1),
 ('сарабєєв юрій миколайович', 5),
 ('благодійна організація "дитячий благодійний фонд "сприяння"', -1),
 ('ардашева олександра іларіонівна', 5),
 ('приватне підприємство "нікатед"', -1),
 ('карпенко ірина віталіївна

In [22]:
firms = [x for (x, y) in dict_find.items() if len(y) > 0] # список усіх знайдених контрагентів-юросіб
set([y[2] for (x, y) in dict_find.items() if x in firms]) # можливі стани юросіб

{'в стані припинення',
 'зареєстровано',
 'зареєстровано, свідоцтво про державну реєстрацію недійсне',
 'припинено',
 'скасовано'}

In [53]:
df_types = pd.DataFrame(list_type, columns=['Founder', 'EDRPOU', 'Founder_part', 'Is_benef'])
#df_types.to_excel('Founded_types.xlsx', index=False)
df_types.tail()

,Founder,EDRPOU,Founder_part,Is_benef
170,міжнародна комерційна компанія bon enterprises...,34055644,1.0,0
171,зарх владислав,43269310,0.0,0
172,карпенко ірина віталіївна,43691977,0.5,1
173,43012721,43691977,0.5,0
174,будяну олександр георгійович,25964747,0.0,1


In [19]:
df_heads = pd.DataFrame(list_head, columns=['Head', 'EDRPOU'])
#df_heads.to_excel('Founded_heads.xlsx', index=False)
df_heads.tail()

,Head,EDRPOU
46,звєрєв сергій валерійович,34055644
47,ардашева олександра іларіонівна,43269310
48,карпенко ірина віталіївна,43691977
49,кручко галина володимирівна,43691977
50,будяну олександр георгійович,25964747


Сформуємо список усіх знайдених контрагентів (вершин графа).

In [23]:
dict_colours = {'припинено': 'red', 'в стані припинення': 'orange', 
                'зареєстровано, свідоцтво про державну реєстрацію недійсне': 'yellow', 
                'зареєстровано': 'green', 'скасовано': 'blue'}

In [25]:
nodes = []
ident = 0
for key, value in dict_find.items():
    ident += 1
    if len(value) > 0: # знайдений контрагент -- юрособа
        edrpou = key
        name = value[1]
        colour = dict_colours[value[2]]
    else: # знайдений контрагент -- фізособа
        edrpou = ''
        name = key
        colour = 'white'
    nodes.append({'id': ident, 'colour': colour, 'name': name, 'edrpou': edrpou})

In [48]:
other = set(df_types['Founder']) - (set([x['name'] for x in nodes]) | set([x['edrpou'] for x in nodes]))
ident = len(nodes)
for item in other:
    ident += 1
    nodes.append({'id': ident, 'colour': 'black', 'name': item, 'edrpou': ''})

Формуємо граф, поки що без зв'язків, тільки з вершинами.

In [49]:
G = {"directed": True, "multigraph": True, "graph": {}, "nodes": nodes, "links": []}

In [50]:
with open('../json/graph.json', 'w', encoding='utf-8') as file:
    json.dump(G, file)

Тепер додаємо до вершин ідентифікатори і формуємо зв'язки між вершинами.

In [51]:
df_nodes = pd.DataFrame(nodes)
df_nodes['value'] = df_nodes[['name', 'edrpou']].apply(
    lambda x: x['edrpou'] if x['edrpou'] != '' else x['name'], axis=1)
df_nodes

,id,colour,name,edrpou,value
0,1,green,"тов ""сигарний дім фортуна""",31382665,31382665
1,2,white,білоус вікторія віталіївна,,білоус вікторія віталіївна
2,3,white,білоус віталій григорійович,,білоус віталій григорійович
3,4,white,єлісєєв олександр,,єлісєєв олександр
4,5,green,"тов ""хіт-трейд""",42275040,42275040
...,...,...,...,...,...
279,280,black,беллкор інвестмен лімітед (bellkor investment ...,,беллкор інвестмен лімітед (bellkor investment ...
280,281,black,"товариство з обмеженою відповідальністю ""рінсв...",,"товариство з обмеженою відповідальністю ""рінсв..."
281,282,black,мялковська-рокотова вікторія олександрівна,,мялковська-рокотова вікторія олександрівна
282,283,black,беллкор інвестмент лімітед (bellkor investment...,,беллкор інвестмент лімітед (bellkor investment...


In [56]:
def relation_type(part):
    if part >= 0.5:
        return 'high'
    elif part >= 10:
        return 'medium'
    elif part > 0:
        return 'small'
    else:
        return 'undefined'

In [55]:
dict_colors = {'benef': 'magenta', 'high': 'red', 'medium': 'yellow', 'small': 'green', 'undefined': 'black', 
               'head': 'blue'}

In [57]:
df_types = df_types.merge(df_nodes[['id', 'value']].rename(columns={'value': 'Founder'}), how='left', 
                          on='Founder').rename(columns={'id': 'source'})
df_types = df_types.merge(df_nodes[['id', 'value']].rename(columns={'value': 'EDRPOU'}), how='left', 
                          on='EDRPOU').rename(columns={'id': 'target'})
df_types['type'] = df_types['Founder_part'].apply(lambda x: relation_type(x))
df_types.tail()

,Founder,EDRPOU,Founder_part,Is_benef,source,target,type
170,міжнародна комерційна компанія bon enterprises...,34055644,1.0,0,277,226,high
171,зарх владислав,43269310,0.0,0,22,232,undefined
172,карпенко ірина віталіївна,43691977,0.5,1,236,235,high
173,43012721,43691977,0.5,0,63,235,high
174,будяну олександр георгійович,25964747,0.0,1,45,259,undefined


In [58]:
df_heads = df_heads.merge(df_nodes[['id', 'value']].rename(columns={'value': 'Head'}), how='left', 
                          on='Head').rename(columns={'id': 'source'})
df_heads = df_heads.merge(df_nodes[['id', 'value']].rename(columns={'value': 'EDRPOU'}), how='left', 
                          on='EDRPOU').rename(columns={'id': 'target'})
df_heads.tail()

,Head,EDRPOU,source,target
46,звєрєв сергій валерійович,34055644,21,226
47,ардашева олександра іларіонівна,43269310,233,232
48,карпенко ірина віталіївна,43691977,236,235
49,кручко галина володимирівна,43691977,25,235
50,будяну олександр георгійович,25964747,45,259


In [59]:
df_rel = df_types[['source', 'target', 'type']]
df_ben = df_types[df_types['Is_benef'] == 1][['source', 'target']].reset_index(drop=True)
df_ben['type'] = 'benef'
df_head = df_heads[['source', 'target']]
df_head['type'] = 'head'
df_type = pd.concat([df_rel, df_ben, df_head], ignore_index=True)
df_type = df_type.sort_values(by=['source', 'target']).reset_index(drop=True)

<ipython-input-59-df857b58dcb3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_head['type'] = 'head'


In [61]:
df_type['type'].value_counts()

small        137
head          51
benef         23
high          21
undefined     17
Name: type, dtype: int64

In [62]:
keys = [0]
key = 0
source = df_type['source'][0]
target = df_type['target'][0]
for i in range(1, len(df_type)):
    source_new = df_type['source'][i]
    target_new = df_type['target'][i]
    if (target_new == target) and (source_new == source):
        key += 1
    else:
        key = 0
    keys.append(key)
    source = source_new
    target = target_new

In [63]:
df_type['color'] = df_type['type'].map(dict_colors)
df_type['key'] = keys
df_type.tail()

,source,target,type,color,key
244,283,18,small,green,0
245,283,47,small,green,0
246,283,50,small,green,0
247,283,59,small,green,0
248,284,30,high,red,0


In [64]:
df_type['key'].value_counts()

0    220
1     27
2      2
Name: key, dtype: int64

In [76]:
links = []
for i in range(len(df_type)): # for JSON, type of integers must be int, not int64!!!
    links.append({'source': int(df_type['source'][i]), 'target': int(df_type['target'][i]), 
                  'color': df_type['color'][i], 'key': int(df_type['key'][i])}) 

In [77]:
G["links"] = links

In [78]:
with open('../json/graph.json', 'w', encoding='utf-8') as file:
    json.dump(G, file)

That's all!!! We have nodes and endes of our graph in JSON. Лишилося тільки додати степені вершин.

In [83]:
G = read_json('../json/graph.json')

In [84]:
G = nodes_degree(G)

In [85]:
write_json(G, '../json/graph.json')

In [39]:
#dict_nodes_jur = dict()
#for i in list(dict_find.keys()):
#    if i.isnumeric():
#        dict_nodes_jur[i] = dict_find[i]

In [ ]:
df_nodes = pd.DataFrame(dict_find_new)#, columns=['Head', 'EDRPOU'])
df_nodes#.to_excel('Fortuna_nodes.xlsx', index=False)